In [1]:
import pandas as pd
import numpy as np
import pylab as pp
import os
import requests
import io
import urllib.request

os.chdir("D:\Stocks\PersianCal")
import PersianCal

In [2]:
os.chdir('D:/Stocks/')
file = 'Codes.csv'
codes = pd.read_csv(file)
# code = str(codes.Codes[code_number])

In [3]:
# code_number = 98
# code = str(codes.Codes[code_number])

code = '70474983732269112'

url = "http://www.tsetmc.com/tsev2/data/Export-txt.aspx?t=i&a=1&b=0&i=" + code
r = requests.get(url, allow_redirects=True).content

# open('7769065543658313.csv', 'wb').write(r.content);

df = pd.read_csv(io.StringIO(r.decode('utf-8')))

In [4]:
## Datetime conversion
df.rename(columns={"<DTYYYYMMDD>": "Date"}, inplace=True)
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')

# Dropping any NaNs
df.dropna(inplace=True)

In [5]:
df['Date'] =pd.to_datetime(df.Date)
df = df.sort_values(by=['Date'])

In [6]:
d = []
for i in range(len(df["Date"])):
    d.append(PersianCal.Christian(str(df["Date"][i])[0:10]).persian_string()) #Tabdil tarikh be Irani

In [7]:
for i in range(len(df["Date"])):
    df["Date"] = df["Date"].replace(df["Date"][i], d[i])

In [8]:
date_list = []
for date in df["Date"]:
    date_list.append(date[:7])
    
Dates = df["Date"].tolist() #kole tarikh ha
myDates = list(dict.fromkeys(date_list)) #Eshterake Tatikh ha

In [9]:
ext = np.zeros((len(myDates), 2)) #[[max], [min]]
ind = np.zeros((len(myDates), 1)).tolist()

#first indicator
for date in Dates:
    if (date[:7] in myDates[0]):
        ind1 = Dates.index(date) #یافتن مکان آخرین روز ماه

ind[0] = ind1

# First max and min:

In [10]:
a = [[], [], [], []]
# names = ["<Open>", "<High>", "<Low>", "<Close>"]
names = ["<FIRST>", "<HIGH>", "<LOW>", "<CLOSE>"]

for i in range(4):
    for j in names:
        a[i] = df[j][0:ind1+1].tolist()

ext[0][0] = max(max(a))
ext[0][1] = min(min(a))

# Calculating all of the extremums:

In [11]:
for i in range(1, len(myDates)):
    for date in Dates:
        if (date[:7] in myDates[i]):
            ind1 = Dates.index(date)
    ind[i] = ind1
    a = [[], [], [], []]
    for ii in range(4):
        for j in names:
            a[ii] = df[j][ind[i-1]+1:ind[i]+1].tolist()
    ext[i][0] = max(max(a))
    ext[i][1] = min(min(a))

In [12]:
# import pylab as pp

# %matplotlib notebook
# import pylab as pp

# pp.plot(DataF["Date"], DataF["Max"], color='skyblue', label = "Max")
# pp.plot(DataF["Date"], DataF["Min"], color='red', label = "Min")
# pp.legend();

In [13]:
def getIndexes(dfObj, value):
    ''' Get index positions of value in dataframe i.e. dfObj.'''
    listOfPos = list()
    # Get bool dataframe with True at positions where the given value exists
    result = dfObj.isin([value])
    # Get list of columns that contains the value
    seriesObj = result.any()
    columnNames = list(seriesObj[seriesObj == True].index)
    # Iterate over list of columns and fetch the rows indexes where value exists
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            listOfPos.append((row, col))
    # Return a list of tuples indicating the positions of value in the dataframe
    return listOfPos

In [14]:
# listOfPositions = getIndexes(df[ind[4]+1:ind[5]+1], ext[5][1])
# print('Index positions of 1835000 in Dataframe: ')
# for i in range(len(listOfPositions)):
#     print('Position ', i, ' (Row index , Column Name) : ', listOfPositions[i])

In [15]:
minDate = []
firstPos = getIndexes(df[0:ind[0]+1], ext[0][1])
minDate.append(df.Date[firstPos[0][0]])

for j in range(len(ext)-1):
    listOfPositions = getIndexes(df[ind[j]+1:ind[j+1]+1], ext[j+1][1])
    for i in range(len(listOfPositions)):
        minDate.append(df.Date[listOfPositions[0][0]])

minDate = list(dict.fromkeys(minDate))

In [16]:
maxDate = []
firstPos2 = getIndexes(df[0:ind[0]+1], ext[0][0])
maxDate.append(df.Date[firstPos2[0][0]])

for j in range(len(ext)-1):
    listOfPositions = getIndexes(df[ind[j]+1:ind[j+1]+1], ext[j+1][0])
    for i in range(len(listOfPositions)):
        maxDate.append(df.Date[listOfPositions[0][0]])
        
maxDate = list(dict.fromkeys(maxDate))

In [17]:
first = []
last = []

for i in range(len(maxDate)):
    if (int(maxDate[i][8:10]) < int(minDate[i][8:10])):
        first.append(ext[i, 0])
        last.append(ext[i, 1])
    else:
        first.append(ext[i, 1])
        last.append(ext[i, 0])

In [20]:
# res = {"Date": myDates,
#        "First": first,
#        "Last": last}

res = {"Date": myDates,
       "First": first,
       "Last": last,
       "Max": ext[:, 0],
       "Min": ext[:, 1]}

resF = pd.DataFrame(res)

In [21]:
os.chdir("D:/")

file_name = str(df['<TICKER>'][1]).replace('*', '')

resF.set_index('Date', inplace=True)
resF.to_csv(file_name + '_output' + '.csv')